## Description
The notebook explores step-by-step process of PLM-based phrase embedding calculcation that is presented in 'plm_emb.py' script

In [2]:
import json
from tqdm import tqdm
import numpy as np
import copy
from transformers import AutoTokenizer, AutoModel
import torch
import os
import argparse
from nltk import sent_tokenize
from functools import reduce
import mmap
from gensim.models.keyedvectors import KeyedVectors
import pickle

In [3]:
from transformers import AutoModel, AutoTokenizer
model_name = "EMBEDDIA/sloberta" 
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of CamembertModel were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import nltk

In [12]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\ILLIA\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\ILLIA\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\ILLIA\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\ILLIA\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\ILLIA\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\Users\ILLIA\AppData\Roaming\nl

True

In [5]:
with open('../Project/data/news/314309.json','r') as f:
    d = json.load(f)
s = d['text']

In [7]:
sentences = sent_tokenize(s.strip().lower().replace(' .', '.'),language='slovene')

In [9]:
sent_toks = [tok for tok in sentences[1].replace('.', ' .').split(' ') if tok != '']

In [11]:
tok_enc = tokenizer([tok.replace('_', ' ') for tok in sent_toks], add_special_tokens=False)['input_ids']

In [13]:
tok_enc

[[1578, 16779, 5716],
 [98, 4322, 99],
 [7006, 55, 6710],
 [1586, 31781, 8688],
 [38],
 [28],
 [17391, 26508, 27820],
 [19896],
 [37],
 [29702, 4856],
 [7187, 3056, 20],
 [6830, 29],
 [37],
 [12151, 31818],
 [13452, 31791],
 [19019, 1838],
 [23, 6710],
 [353, 213, 36],
 [82],
 [14726, 31818],
 [13452],
 [1666]]

In [14]:
tokenizer.cls_token_id

5

In [15]:
[1]+(np.cumsum([len(ids) for ids in tok_enc])+1).tolist()

[1,
 4,
 7,
 10,
 13,
 14,
 15,
 18,
 19,
 20,
 22,
 25,
 27,
 28,
 30,
 32,
 34,
 36,
 39,
 40,
 42,
 43,
 44]

In [16]:
indices = [1] + (np.cumsum([len(ids) for ids in tok_enc]) + 1).tolist()

In [27]:
tok_enc

[[1578, 16779, 5716],
 [98, 4322, 99],
 [7006, 55, 6710],
 [1586, 31781, 8688],
 [38],
 [28],
 [17391, 26508, 27820],
 [19896],
 [37],
 [29702, 4856],
 [7187, 3056, 20],
 [6830, 29],
 [37],
 [12151, 31818],
 [13452, 31791],
 [19019, 1838],
 [23, 6710],
 [353, 213, 36],
 [82],
 [14726, 31818],
 [13452],
 [1666]]

In [17]:
flat_ids = [tokenizer.cls_token_id] + reduce(lambda x, y: x + y, tok_enc, []) + [tokenizer.sep_token_id]

In [18]:
flat_ids

[5,
 1578,
 16779,
 5716,
 98,
 4322,
 99,
 7006,
 55,
 6710,
 1586,
 31781,
 8688,
 38,
 28,
 17391,
 26508,
 27820,
 19896,
 37,
 29702,
 4856,
 7187,
 3056,
 20,
 6830,
 29,
 37,
 12151,
 31818,
 13452,
 31791,
 19019,
 1838,
 23,
 6710,
 353,
 213,
 36,
 82,
 14726,
 31818,
 13452,
 1666,
 6]

In [19]:
len(flat_ids)

45

In [20]:
vocab={}
inv_vocab={}

In [21]:
for tok in sent_toks:
    if tok not in vocab:
        vocab[tok] = len(vocab)
        inv_vocab[vocab[tok]] = tok

In [24]:
ses=[]

In [25]:
ses.append((flat_ids, [vocab[tok] for tok in sent_toks], indices))

In [26]:
ses

[([5,
   1578,
   16779,
   5716,
   98,
   4322,
   99,
   7006,
   55,
   6710,
   1586,
   31781,
   8688,
   38,
   28,
   17391,
   26508,
   27820,
   19896,
   37,
   29702,
   4856,
   7187,
   3056,
   20,
   6830,
   29,
   37,
   12151,
   31818,
   13452,
   31791,
   19019,
   1838,
   23,
   6710,
   353,
   213,
   36,
   82,
   14726,
   31818,
   13452,
   1666,
   6],
  [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   8,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20],
  [1,
   4,
   7,
   10,
   13,
   14,
   15,
   18,
   19,
   20,
   22,
   25,
   27,
   28,
   30,
   32,
   34,
   36,
   39,
   40,
   42,
   43,
   44])]

In [28]:
vocab = {}
inv_vocab = {}
sentences = []
src_dir = '../Project/data/news/'
for file in tqdm(os.listdir(src_dir),total=len(os.listdir(src_dir))):
    with open(os.path.join(src_dir,file),'r') as f:
        d = json.load(f)
    if 'text' in d:
        s = d['text']
        for sent in sent_tokenize(s.strip().lower().replace(' .', '.'),language='slovene'):
            sent_toks = [tok for tok in sent.replace('.', ' .').split(' ') if tok != '']
            tok_enc = tokenizer([tok.replace('_', ' ') for tok in sent_toks], add_special_tokens=False)['input_ids']
            indices = [1] + (np.cumsum([len(ids) for ids in tok_enc]) + 1).tolist()
            flat_ids = [tokenizer.cls_token_id] + reduce(lambda x, y: x + y, tok_enc, []) + [tokenizer.sep_token_id]
            if len(flat_ids) > 512: continue
            for tok in sent_toks:
                if tok not in vocab:
                    vocab[tok] = len(vocab)
                    inv_vocab[vocab[tok]] = tok
            sentences.append((flat_ids, [vocab[tok] for tok in sent_toks], indices))

100%|██████████| 721/721 [00:09<00:00, 76.18it/s]


In [29]:
batch_size = 32
iterations = int(len(sentences)/batch_size) + (0 if len(sentences) % batch_size == 0 else 1)
phrase_div = np.zeros((len(vocab), 1))
phrase_emb = np.zeros((len(vocab), 768))

In [30]:
start = 0 * batch_size
end = min((0+1)*batch_size, len(sentences))
start, end

(0, 32)

In [31]:
batch_ids = [ids for ids,_,_ in sentences[start:end]]
batch_max_length = max(len(ids) for ids in batch_ids)

In [42]:
print(len(batch_ids)),print(batch_ids[0])

32
[5, 4269, 12000, 31791, 3245, 1666, 4700, 100, 97, 31808, 233, 844, 68, 72, 735, 4015, 9056, 100, 59, 144, 238, 31791, 224, 861, 31791, 9749, 238, 31808, 13, 26047, 4235, 1723, 34, 8834, 23, 6796, 195, 13, 9203, 1236, 31791, 13, 4704, 2615, 2192, 9799, 7375, 78, 28785, 282, 45, 227, 78, 72, 4382, 1849, 34, 31791, 67, 72, 12826, 18762, 4299, 33, 4205, 2667, 17943, 33, 366, 1593, 4302, 175, 10192, 3931, 34, 3999, 18, 5342, 11708, 31791, 3443, 12230, 4015, 10209, 33, 26075, 173, 5688, 5448, 28785, 6796, 13, 18282, 2840, 12131, 9713, 3999, 89, 5986, 1666, 6]


(None, None)

In [37]:
ids = torch.tensor([ids + [0 for _ in range(batch_max_length - len(ids))] for ids in batch_ids]).long()
masks = (ids != 0).long()
ids.shape

torch.Size([32, 101])

In [40]:
ids[1]

tensor([    5, 17868,  1763,    68,   631,  3452, 15877,   270,  3999,    89,
         5986,  8018,  1471,  3464, 28004,    33,    23, 14769,    79,   265,
        23811,  3833, 12131,  1684,   283,   407,    18, 12799, 31840,  1320,
        11769,  7375,  3700,  1236,  7752,    10, 31791,   157,    38, 26075,
          173,  5688,  3952, 13044,   642,  1845,  1090, 31791,    67,  1319,
         2665,  1666,     6,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0])

In [44]:
batch_final_layer = model(ids, masks)[0]
batch_final_layer

tensor([[[ 1.1348e-01, -7.0428e-02, -3.2933e-02,  ..., -9.5881e-02,
           8.2250e-02,  1.3673e-02],
         [-4.5574e-01,  5.2130e-02, -5.3951e-01,  ..., -2.7226e-01,
          -2.2225e-01, -6.6631e-01],
         [-3.9055e-01,  3.9250e-02,  3.0773e-01,  ..., -2.7218e-01,
          -2.0976e-01, -1.9855e-01],
         ...,
         [-2.9948e-01,  1.2349e-01,  9.0223e-02,  ..., -3.5650e-01,
           2.6992e-01, -3.1598e-01],
         [-1.5737e-01, -7.9548e-02,  1.3892e-01,  ..., -1.7102e-02,
          -3.4802e-02, -3.3593e-01],
         [-7.1166e-02,  8.8937e-02,  1.1284e-01,  ...,  2.2654e-01,
           2.5839e-01, -4.9933e-01]],

        [[ 1.7489e-01, -8.3358e-02, -1.0271e-02,  ..., -4.3773e-02,
           2.7871e-02, -1.1671e-01],
         [-5.7650e-02, -4.0602e-02,  2.5706e-01,  ..., -1.8942e-01,
           2.2271e-01, -8.8724e-02],
         [-8.7885e-02,  1.5177e-01,  5.4022e-02,  ..., -2.3902e-02,
          -2.5731e-03, -1.2130e-01],
         ...,
         [ 1.8795e-01, -8

In [45]:
batch_final_layer.shape

torch.Size([32, 101, 768])

In [51]:
print(sentences[1][0])

[5, 17868, 1763, 68, 631, 3452, 15877, 270, 3999, 89, 5986, 8018, 1471, 3464, 28004, 33, 23, 14769, 79, 265, 23811, 3833, 12131, 1684, 283, 407, 18, 12799, 31840, 1320, 11769, 7375, 3700, 1236, 7752, 10, 31791, 157, 38, 26075, 173, 5688, 3952, 13044, 642, 1845, 1090, 31791, 67, 1319, 2665, 1666, 6]


In [52]:
print(sentences[1][1])

[56, 7, 57, 58, 59, 54, 55, 60, 61, 62, 34, 19, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 48, 77, 78, 79, 80, 81, 30, 82, 83, 2]


In [53]:
print(sentences[1][2])

[1, 3, 4, 5, 6, 8, 10, 11, 13, 14, 15, 16, 17, 19, 20, 22, 25, 27, 29, 30, 31, 32, 33, 34, 37, 38, 39, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52]


In [48]:
for final_layer, (_,sent_ids,indices) in zip(batch_final_layer, sentences[start:end]):
    print(final_layer.shape, len(sent_ids), len(indices))

torch.Size([101, 768]) 64 65
torch.Size([101, 768]) 36 37
torch.Size([101, 768]) 47 48
torch.Size([101, 768]) 34 35
torch.Size([101, 768]) 32 33
torch.Size([101, 768]) 39 40
torch.Size([101, 768]) 37 38
torch.Size([101, 768]) 42 43
torch.Size([101, 768]) 28 29
torch.Size([101, 768]) 26 27
torch.Size([101, 768]) 37 38
torch.Size([101, 768]) 17 18
torch.Size([101, 768]) 17 18
torch.Size([101, 768]) 28 29
torch.Size([101, 768]) 19 20
torch.Size([101, 768]) 21 22
torch.Size([101, 768]) 30 31
torch.Size([101, 768]) 24 25
torch.Size([101, 768]) 30 31
torch.Size([101, 768]) 29 30
torch.Size([101, 768]) 20 21
torch.Size([101, 768]) 19 20
torch.Size([101, 768]) 27 28
torch.Size([101, 768]) 16 17
torch.Size([101, 768]) 13 14
torch.Size([101, 768]) 38 39
torch.Size([101, 768]) 35 36
torch.Size([101, 768]) 44 45
torch.Size([101, 768]) 22 23
torch.Size([101, 768]) 10 11
torch.Size([101, 768]) 21 22
torch.Size([101, 768]) 16 17


In [55]:
for i in tqdm(range(iterations)):
    start = i * batch_size
    end = min((i+1)*batch_size, len(sentences))
    batch_ids = [ids for ids,_,_ in sentences[start:end]]
    batch_max_length = max(len(ids) for ids in batch_ids)
    ids = torch.tensor([ids + [0 for _ in range(batch_max_length - len(ids))] for ids in batch_ids]).long()
    masks = (ids != 0).long()
    #ids = ids.to(device)
    #masks = masks.to(device)
    with torch.no_grad():
        batch_final_layer = model(ids, masks)[0]
    for final_layer, (_,sent_ids,indices) in zip(batch_final_layer, sentences[start:end]):
        for idx in range(len(sent_ids)):
            tok_id = sent_ids[idx]
            phrase_emb[tok_id] += np.mean(final_layer[indices[idx]:indices[idx+1]].cpu().numpy(), axis=0)
            phrase_div[tok_id] += 1

100%|██████████| 56/56 [08:57<00:00,  9.61s/it]


In [56]:
ave_phrase_emb = phrase_emb / phrase_div

kv = KeyedVectors(768)
kv.add_vectors([inv_vocab[i] for i in range(len(vocab))], ave_phrase_emb)
kv.save(f'example_sloberta.kv')

In [57]:
len(kv)

13560